<span style="color: black; font-family: Babas; font-size: 2em"> El primer paso es importar todos los modulos necesarios para el funcionamiento del programa</span>

In [1]:
import gdal
import matplotlib.pyplot as plt
import numpy as np
from gdalconst import *

ModuleNotFoundError: No module named 'gdal'

<span style="color: black; font-family: Babas; font-size: 2em"> Se definen todas las constantes a usar</span>(La demostracion y explicacion de que son estas constantes puede ser encontrada haciendo click [aqui](https://www.google.com))

In [36]:
UCCj_banda10=0.006822 ;lamdba_banda10= 8.291 ;K1_banda10=3040.136402 ;K2_banda10=1735.337945
UCCj_banda11=0.006780 ;lamdba_banda11=8.634 ;K1_banda11=2482.375199 ;K2_banda11=1666.398761
UCCj_banda12=0.006590 ;lamdba_banda12=9.075 ;K1_banda12=1935.060183 ;K2_banda12=1585.420044
UCCj_banda13=0.005693 ;lamdba_banda13=10.657 ;K1_banda13=866.468575 ;K2_banda13=1350.069147
UCCj_banda14=0.005225 ;lamdba_banda14=11.318 ;K1_banda14=641.326517 ;K2_banda14=1271.221673

<span style="color: black; font-family: Babas; font-size: 2em"> Luego accedemos al directorio desde donde se correra el programa usando comando de bash y verificamos con ls</span>

In [37]:
%%bash
cd /run/media/cbarrios/NuevoVol/descargas/
ls *.hdf

AST_L1T_00302082015025425_20170702214312_25641.hdf
AST_L1T_00302142014150356_20150619212535_83261.hdf
AST_L1T_00302252012024751_20170720020559_2254.hdf
AST_L1T_00304022011025346_20170723190244_15301.hdf
AST_L1T_00306272010150355_20150602001041_5325.hdf
AST_L1T_00307132016024852_20170626061157_20158.hdf
AST_L1T_00312212013145751_20150619015916_100229.hdf


<span style="color: black; font-family: Babas; font-size: 2em">Luego cargamos los archivos .hdf escogiendo cual sera el nombre del archivo y usando el modulo gdal , para luego ver los sub datos del archivo hdf</span>(lo comentado lo hace automatico si en la misma celda)

In [38]:
path="/run/media/cbarrios/NuevoVol/descargas/" 
#A=np.array([])
#for file in os.listdir(path):
  #if file.startswith("Ariel"):
    #A=np.append(A,file)
    #se usaria filename[k] vez de filename en un loop junto con los pasos posteriores en la siguiente celda 
filename="AST_L1T_00306272010150355_20150602001041_5325.hdf" 
Mi_Data=gdal.Open(path+filename,GA_ReadOnly)
Mi_Data.GetSubDatasets()

[('HDF4_EOS:EOS_SWATH:"/run/media/cbarrios/NuevoVol/descargas/AST_L1T_00306272010150355_20150602001041_5325.hdf":VNIR_Swath:ImageData2',
  '[4861x5527] ImageData2 VNIR_Swath (8-bit unsigned integer)'),
 ('HDF4_EOS:EOS_SWATH:"/run/media/cbarrios/NuevoVol/descargas/AST_L1T_00306272010150355_20150602001041_5325.hdf":VNIR_Swath:ImageData1',
  '[4861x5527] ImageData1 VNIR_Swath (8-bit unsigned integer)'),
 ('HDF4_EOS:EOS_SWATH:"/run/media/cbarrios/NuevoVol/descargas/AST_L1T_00306272010150355_20150602001041_5325.hdf":VNIR_Swath:ImageData3N',
  '[4861x5527] ImageData3N VNIR_Swath (8-bit unsigned integer)'),
 ('HDF4_EOS:EOS_SWATH:"/run/media/cbarrios/NuevoVol/descargas/AST_L1T_00306272010150355_20150602001041_5325.hdf":TIR_Swath:ImageData10',
  '[811x922] ImageData10 TIR_Swath (16-bit unsigned integer)'),
 ('HDF4_EOS:EOS_SWATH:"/run/media/cbarrios/NuevoVol/descargas/AST_L1T_00306272010150355_20150602001041_5325.hdf":TIR_Swath:ImageData11',
  '[811x922] ImageData11 TIR_Swath (16-bit unsigned in

<span style="color: black; font-family: Babas; font-size: 2em"> Luego guardamos la banda termal a usar en este caso usaremos la banda 11</span>(en versiones posteriores del notebook esto pudo ser automatizado con inputs)

In [39]:
Sub_data=Mi_Data.GetSubDatasets()[4][0]

<span style="color: black; font-family: Babas; font-size: 2em"> Cargamos el subdato con gdal en una nueva variable</span>

In [40]:
Banda11=gdal.Open(Sub_data,GA_ReadOnly)

<span style="color: black; font-family: Babas; font-size: 2em"> Transformamos el raster en un array para realizar los calculos</span>

In [41]:
Banda11_array=Banda11.ReadAsArray()
Banda11_array

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint16)

<span style="color: black; font-family: Babas; font-size: 2em"> Se plotea con imshow y se añade una barra de color para visualizar nuestro array extraido del raster</span>

In [42]:
plt.imshow(Banda11_array)
plt.colorbar()
plt.show()

<span style="color: black; font-family: Babas; font-size: 2em"> Obtenemos la forma del array y definimos uno nuevo con las mismas dimensiones donde se almacenara nuestro primer calculo </span>

In [43]:
x,y=Banda11_array.shape
Banda11_array_conv=np.empty((x,y))

<span style="color: black; font-family: Babas; font-size: 2em"> Se realiza la conversion de numeros digitales a radianza espectral</span> (explicacion [aqui](https://www.google.com)) ademas se calcula el tiempo que tarda en ejecutarse la celda

In [44]:
#%%timeit 
for i in range(0,x):
    for j in range(0,y):
        Banda11_array_conv[i,j]=(Banda11_array[i,j]-1)*UCCj_banda11


1 loops, best of 3: 11.6 s per loop


<span style="color: black; font-family: Babas; font-size: 2em"> Visualizamos el cambio en nuestro raster</span> 

In [45]:
plt.imshow(Banda11_array_conv)
plt.colorbar()
plt.show()

<span style="color: black; font-family: Babas; font-size: 2em"> Definimos un nuevo array donde guardaremos nuestro segundo calculo y realizamos el calculo</span>(el error que vemos se debe a los valores sin medicion) 

In [48]:
#%%timeit
Banda11_T=np.zeros((x,y))
for i in range(0,x):
    for j in range(0,y):
        Banda11_T[i,j]=((K2_banda11)/np.log((K1_banda11/Banda11_array_conv[i,j])+1))-273.15

-c:4: RuntimeWarning: invalid value encountered in log


<span style="color: black; font-family: Babas; font-size: 2em"> De nuevo visualizamos </span> 

In [50]:
plt.imshow(Banda11_T)
plt.colorbar()
plt.show()

<span style="color: black; font-family: Babas; font-size: 2em"> Y por ultimo Guardamos un raster con nuestra nueva data</span> 

In [ ]:
band1 = Banda10.GetRasterBand(1)
geotransform = Banda10.GetGeoTransform()
geoproj = Banda10.GetProjection()
band1data =Banda10_T
format = "GTiff"
driver = gdal.GetDriverByName(format)
dst_datatype = gdal.GDT_Float32
dst_ds = driver.Create(filename+"tiff",y,x,1,dst_datatype)
dst_ds.SetGeoTransform(geotransform)
dst_ds.SetProjection(geoproj)
dst_ds.GetRasterBand(1).WriteArray(Banda10_T)
dst_ds=None